In [ ]:
import cv2
import skimage.io as io
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import helper as helper
from dataset import *


In [ ]:
label_path='../labels/coco/labels/val2017/000000007108.txt'
with open(label_path) as box:
    box=box.read()
    box=pd.DataFrame([x.split() for x in box.rstrip('\n').split('\n')],columns=['class','xc','yc','w','h'])

b= box.values.astype(np.float32)
bboxes=b
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
conf=torch.ones([b.shape[0],1])
boxes=torch.cat((b.T[1:],conf.T,one_hot_target.T)).T

img_path='../images/val2017/000000007108.jpg'
img=cv2.imread(img_path)[:,:,::-1] 

sample={'images': img,
        'boxes': boxes}

(h,w,c)=img.shape

bboxes[:,1]=bboxes[:,1]*w
bboxes[:,2]=bboxes[:,2]*h
bboxes[:,3]=bboxes[:,3]*w
bboxes[:,4]=bboxes[:,4]*h
bboxes[:,1]=bboxes[:,1]-bboxes[:,3]/2
bboxes[:,2]=bboxes[:,2]-bboxes[:,4]/2
bboxes[:,3]=bboxes[:,1]+bboxes[:,3]
bboxes[:,4]=bboxes[:,2]+bboxes[:,4]
print(bboxes)


In [ ]:
import imgaug as ia
from imgaug import parameters as iap
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


image = sample['images']

bbs = BoundingBoxesOnImage([
    BoundingBox(x1=b[1], y1=b[2], x2=b[3], y2=b[4], label=b[0]) for b in bboxes], shape=image.shape)

seq = iaa.Sequential([
    iaa.LinearContrast(alpha=(0.1, 1.9)),
    iaa.ShearX((-30, 30)),
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Grayscale(alpha=(0.1, 0.9)),
    iaa.Sometimes(
        0.5,
        iaa.HistogramEqualization(),
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.Solarize(0.5, threshold=(0, 256)),
    # Strengthen or weaken the contrast in each image.
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        translate_px={"x": (-150, 150)},
        rotate=(-30, 30)
    )
], random_order=True)

# Augment BBs and images.
image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

# print coordinates before/after augmentation (see below)
# use .x1_int, .y_int, ... to get integer coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
        i,
        before.x1, before.y1, before.x2, before.y2,
        after.x1, after.y1, after.x2, after.y2)
    )

# image with BBs before/after augmentation (shown below)
# image_before = bbs.draw_on_image(image, size=2)
# image_after = bbs_aug.draw_on_image(image_aug, size=2, color=[0, 0, 255])

In [ ]:
new_bboxes=bbs_aug.to_xyxy_array()
bbs_aug=bbs_aug.remove_out_of_image()

ia.imshow(bbs_aug.draw_on_image(image_aug, size=2))

labels=np.array([[box.label for box in bbs_aug.bounding_boxes]]).T
new_bboxes=bbs_aug.to_xyxy_array()
np.hstack((labels,new_bboxes))


In [ ]:
ia.imshow(bbs.draw_on_image(image, size=2))

In [ ]:
box=open("../detections/000000007108.txt")
box=box.read()
box=pd.DataFrame([x.split() for x in box.rstrip('\n').split('\n')],columns=['class','conf','xc','yc','w','h'])
b= box.values.astype(np.float32)
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
objectness = b.T[1].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
# conf=torch.ones([b.shape[0],1])

boxes=torch.cat((b.T[2:],objectness.T,one_hot_target.T)).T
print(boxes.shape)
# plt.imshow(draw_rect(sample['images'], boxes))
# plt.savefig('elephants.png')

In [ ]:
import torchvision.ops.boxes as nms_box
import util as util
abs_boxes=util.get_abs_coord(boxes[:,0:4].unsqueeze(0)).squeeze(1)
print(abs_boxes.shape)
fb=nms_box.nms(abs_boxes,boxes[:,4].cuda(),0.5)
print(nms_box.box_iou(abs_boxes,abs_boxes))
print(fb)

In [ ]:
test1=torch.tensor([[15.0,25.0,30.0,50.0],[20.0,30.0,40.0,50.0]])
test2=torch.tensor([[15.0,25.0,35.0,45.0],[20.0,30.0,40.0,50.0]])

fb=nms_box.nms(test1,torch.tensor([0.9,0.9]),0.3)
print(nms_box.box_iou(boxes[:,0:4],boxes[:,0:4]))
print(fb)

In [ ]:
diff=1
import random
print(random.uniform(1,2))
def shear(sample):
    
    img=sample['images']
    bboxes=sample['boxes']
    shear_factor = 0.6

    w,h = img.shape[1], img.shape[0]
    
    if shear_factor < 0:
        sample = RandomHorizontalFlip(-1)(sample)
        img=sample['images']
        bboxes=sample['boxes']

    
    M = np.array([[1, abs(shear_factor), 0],[0,1,0]])

    nW =  img.shape[1] + abs(shear_factor*img.shape[0])

    bboxes[:,0] += bboxes[:,1]* abs(torch.tan(torch.tensor([shear_factor])))
    bboxes[:,2] += bboxes[:,3]* abs(torch.tan(torch.tensor([shear_factor])))
    


    img = cv2.warpAffine(img, M, (int(nW), img.shape[0]))


#     if shear_factor < 0:
#         sample = RandomHorizontalFlip(-1)(sample)
#         img=sample['images']
#         bboxes=sample['boxes']
#         print(bboxes)
#     plt.imshow(draw_rect(img, bboxes))

    img = cv2.resize(img, (w,h))

    scale_factor_x = nW / w
    print(scale_factor_x)
    bboxes[:,:4] /= torch.tensor([scale_factor_x, 1, scale_factor_x, 1])
    print(bboxes)
    return img, bboxes

In [ ]:
img, bboxes = shear(sample)

plt.imshow(draw_rect(img, bboxes))




In [ ]:
import cv2 
import pickle as pkl
import numpy as np 
import matplotlib.pyplot as plt

def draw_rect(im, cords, color = None):
    """Draw the rectangle on the image
    
    Parameters
    ----------
    
    im : numpy.ndarray
        numpy image 
    
    cords: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the 
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`
        
    Returns
    -------
    
    numpy.ndarray
        numpy image with bounding boxes drawn on it
        
    """
    
    im = im.copy()
    h=im.shape[0]
    w=im.shape[1]
    cords[:,0]=cords[:,0]*w
    cords[:,1]=cords[:,1]*h
    cords[:,2]=cords[:,2]*w
    cords[:,3]=cords[:,3]*h
    if not color:
        color = [255,255,255]
    for cord in cords:
        pt1, pt2 = (cord[0]-cord[2]/2, cord[1]-cord[3]/2) , (cord[0]+cord[2]/2, cord[1]+cord[3]/2)
        
#         pt1, pt2 = (cord[0], cord[1]) , (cord[2], cord[3])
        
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
    
        im = cv2.rectangle(im.copy(), pt1, pt2, color, int(max(im.shape[:2])/200))
    return im

In [ ]:

import matplotlib.pyplot as plt

translate=RandomTranslate()
flip=RandomVerticalFlip()

rotate = RandomRotate(20) 
sample=flip(sample)
sample=translate(sample)
sample = rotate(sample)

plt.imshow(draw_rect(img, bboxes))

In [ ]:
img, bboxes = rotate(sample)

plt.imshow(draw_rect(img, bboxes))


In [ ]:
print(helper.get_corners(bboxes))

In [ ]:
import torch

a=torch.tensor([[[1,2,2,4,0,0,0,1],[2,3,5,3,0,0,1,0]]])
print(a.shape)
b=((a[:,:,4:]==1).nonzero())
b=b[:,-1].unsqueeze(0).unsqueeze(-1)
print(b.shape)
new=a[:,:,:4]
print(new.shape)
new=torch.cat((new.T,b.T)).T
print(new)
sorted_pred=torch.sort(new[0,:,4],descending=False)
print(sorted_pred)

In [ ]:
b = box.values.astype(np.float32)
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
conf=torch.ones([b.shape[0],1])
boxes=torch.cat((b.T[1:],conf.T,one_hot_target.T)).T




In [ ]:
file_name='images/train2017.zip'
with ZipFile(file_name, 'r') as zip:
    image=zip.read('train2017/000000408542.jpg')
    img = cv2.imdecode(np.frombuffer(image, np.uint8),1)
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    io.imshow(im_rgb)
    io.imsave('fig.png',im_rgb)

In [ ]:
df=pd.read_csv('pointers/train2017.txt',names=['filename'])

In [ ]:
df['box']=df['filename'].apply(lambda x: 'coco/labels/'+x.split('.')[0]+'.txt')

In [ ]:
df['box']

In [ ]:
from dataset import *
import timeit 
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import time
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import test as tester
import sys
import timeit
import torch.autograd
import helper as helper
from torch.utils.tensorboard import SummaryWriter
import pandas as pd



net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.to(device='cuda')
cx_cy=net.cx_cy.to(device='cuda')
stride=net.stride.to(device='cuda')


hyperparameters={'lr':0.0001,
                 'batch_size':16,
                 'weight_decay':0.001,
                 'momentum':0.9,
                 'optimizer':'sgd',
                 'alpha':0.5,
                 'gamma':0,
                 'lcoord':5,
                 'lno_obj':1,
                 'iou_type':(1,0,0),#(GIoU,DIoU,CIoU) default is 0,0,0 for iou
                 'iou_ignore_thresh':0.5,
                 'tfidf':True,
                 'idf_weights':True,
                 'tfidf_col_names':['obj_freq','obj_freq','obj_freq','obj_freq','softmax'], #default is ['obj_freq/img_freq','area','xc','yc','softmax']-->[class_weights,scale_weights,xweights,yweights,softmax/no_softmax]
                 'augment':False,
                 'workers':4,
                 'path':'tes_iou',
                 'reduction':'sum'}

print(hyperparameters)

if (hyperparameters['idf_weights']==True):
    hyperparameters['idf_weights']=pd.read_csv('../idf.csv')
else:
    hyperparameters['idf_weights']=False
        
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../'+hyperparameters['path']
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we https://pytorch.org/docs/stable/data.html#torch.utils.data.Datasetare on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if (torch.cuda.device_count() > 1):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
        model.load_state_dict(weights)
    else:
        model=net
        model.to(device)
        model.load_state_dict(weights)
        
except FileNotFoundError: 
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if (torch.cuda.device_count() > 1):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
    else:
        model=net
        
if hyperparameters['augment']==True:
    transformed_dataset=Coco(partition='train',transform=transforms.Compose([Augment(),ResizeToTensor(inp_dim)]))
else:
    transformed_dataset=Coco(partition='train',transform=transforms.Compose([ResizeToTensor(inp_dim)]))
    


writer = SummaryWriter('../results/'+hyperparameters['path'])
dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=hyperparameters['batch_size']
dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.my_collate, num_workers=hyperparameters['workers'])


if hyperparameters['optimizer']=='sgd':
    optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
elif hyperparameters['otimizer']=='adam':
    optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])

lambda1 = lambda epoch: 0.95**epoch
scheduler=optim.lr_scheduler.LambdaLR(optimizer, lambda1, last_epoch=-1)
mAP_max=0
epochs=50
total_loss=0
write=0
misses=0
avg_iou=0
for e in range(epochs):
    prg_counter=0
    train_counter=0
    total_loss=0
    avg_iou=0
    avg_infs=0
    avg_conf=0
    avg_no_conf=0
    avg_pos=0
    avg_neg=0
    print("\n epoch "+str(e))
    misses=0
    for images,targets,img_names in dataloader:
        optimizer.zero_grad()
        targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
        images=images.cuda()
        raw_pred = model(images, torch.cuda.is_available())
        raw_pred=helper.expand_predictions(raw_pred,mask)
        true_pred=util.transform(raw_pred.clone(),anchors,offset,strd)
        targets=targets.unsqueeze(-3)
        targets=targets.cuda()
        iou_mask,noobj_mask=util.get_responsible_masks(true_pred,targets,offset,strd,mask,inp_dim,hyperparameters)
        iou1=torch.diag(util.bbox_iou(util.get_abs_coord(true_pred[iou_mask.T,:].unsqueeze(-3)),util.transpose_target(get_abs_coord(targets)*inp_dim),hyperparameters['iou_type']))
        iou=iou1.mean().item()
        noobj_box=raw_pred[:,:,4:5].clone()
        noobj_box=helper.uncollapse(noobj_box,mask)
        noobj_mask=helper.uncollapse(noobj_mask.T.unsqueeze(-1),mask)
        
        
        conf=raw_pred[iou_mask.T,:][:,4:5].mean().item()
        class_mask=targets[:,:,5:].type(torch.BoolTensor).squeeze(0)
        if(iou_mask.sum()==class_mask.shape[0]):
            pos_class=raw_pred[iou_mask.T,:][:,5:][class_mask].mean().item()
            neg_class=raw_pred[iou_mask.T,:][:,5:][~class_mask].mean().item()
        else:
            pos_class=0
            neg_class=0
            
        noobj_box=noobj_box[noobj_mask]
        no_obj_conf=noobj_box.mean().item()
        
        raw_pred=raw_pred[iou_mask.T,:]
        anchors=anchors[iou_mask.T,:]
        offset=offset[iou_mask.T,:]
        strd=strd[iou_mask.T,:]
        
        if(strd.shape[0]==sum(mask)):#this means that iou_mask failed and was all true, because max of zeros is true for all lenght of mask strd
            targets[:,:,0:4]=targets[:,:,0:4]*inp_dim
            targets=targets.squeeze(0)
            if hyperparameters['iou_type']==(0,0,0):
                targets[:,0:4]=util.transform_groundtruth(targets,anchors,offset,strd)
                loss=util.yolo_loss(raw_pred,targets,noobj_box,mask,hyperparameters)
                loss.backward()
                optimizer.step()
            else:
                raw_pred=util.transform(raw_pred.unsqueeze(0),anchors.unsqueeze(0),offset.unsqueeze(0),strd.unsqueeze(0)).squeeze(0)
                loss=util.yolo_loss(raw_pred,targets,noobj_box,mask,hyperparameters)
                loss.backward()
                optimizer.step()
            
            avg_conf=avg_conf+conf
            avg_no_conf=avg_no_conf+no_obj_conf
            avg_pos=avg_pos+pos_class
            avg_neg=avg_neg+neg_class
            total_loss=total_loss+loss.item()
            avg_iou=avg_iou+iou
            sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%' ' L:'+ str(loss.item()))
            sys.stdout.write(' IoU:' +str(iou)+' pob:'+str(conf)+ ' nob:'+str(no_obj_conf))
            sys.stdout.write(' PCls:' +str(pos_class)+' ncls:'+str(neg_class))
            sys.stdout.flush()
            prg_counter=prg_counter+1
            train_counter=train_counter+1
        else:
            print('missed')
            print(strd.shape[0])
            prg_counter=prg_counter+1
        iou1=iou1.tolist()
        iou_per_class= [0] * 80
        averager_per_class=[1] * 80
        for i,el in enumerate(iou1):
            name=targets[i,5:].max(0)[1].cpu().detach().numpy()
            iou_per_class[name]=iou_per_class[name]+el
            averager_per_class[name]=averager_per_class[name]+1
        for i in range(80):
            if (iou_per_class[i]!=0):
                if i<40:
                    writer.add_scalar('Iou0/'+str(i), iou_per_class[i]/averager_per_class[i], train_counter)
                else:
                    writer.add_scalar('Iou1/'+str(i), iou_per_class[i]/averager_per_class[i], train_counter)
        writer.add_scalar('AvLoss/train', total_loss/train_counter, train_counter)

        
        writer.add_scalar('AvIoU/train', avg_iou/train_counter, train_counter)

        writer.add_scalar('AvPConf/train', avg_conf/train_counter, train_counter)
        
        writer.add_scalar('AvNConf/train', avg_no_conf/train_counter, train_counter)
        
        writer.add_scalar('AvClass/train', avg_pos/train_counter, train_counter)
        
        writer.add_scalar('AvNClass/train', avg_neg/train_counter, train_counter)
        
        del loss, raw_pred, targets, true_pred, images,iou,noobj_box,conf,iou1
        torch.cuda.empty_cache()
    if misses>0:
        break
#     torch.save(model.state_dict(), PATH)
#     writer.add_scalar('Loss/train', total_loss/train_counter, e)
#     writer.add_scalar('AIoU/train', avg_iou/train_counter, e)
#     writer.add_scalar('PConf/train', avg_conf/train_counter, e)
#     writer.add_scalar('NConf/train', avg_no_conf/train_counter, e)
#     writer.add_scalar('PClass/train', avg_pos/train_counter, e)
#     writer.add_scalar('NClass/train', avg_neg/train_counter, e)
    print('\ntotal number of misses is ' + str(misses))
    print('\n total average loss is '+str(total_loss/train_counter))
    print('\n total average iou is '+str(avg_iou/train_counter))
    


{'lr': 0.0001, 'batch_size': 16, 'weight_decay': 0.001, 'momentum': 0.9, 'optimizer': 'sgd', 'alpha': 0.5, 'gamma': 0, 'lcoord': 5, 'lno_obj': 1, 'iou_type': (1, 0, 0), 'iou_ignore_thresh': 0.5, 'tfidf': True, 'idf_weights': True, 'tfidf_col_names': ['obj_freq', 'obj_freq', 'obj_freq', 'obj_freq', 'softmax'], 'augment': False, 'workers': 4, 'path': 'tes_iou', 'reduction': 'sum'}
cuda:0
Using  2 GPUs!
Length of dataset is 118287


 epoch 0


/users/konsa15/.conda/envs/faster_rcnn/lib/python3.8/site-packages/torch/nn/functional.py:2503: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


Pgr:3.922662676371875% L:2.475751278524398 IoU:0.7354631423950195 pob:0.4225219786167145 nob:0.0005686893709935248 PCls:0.9350520968437195 ncls:0.000822125235572457356247

In [ ]:

        iou1=torch.diag(util.bbox_iou(util.get_abs_coord(true_pred[iou_mask.T,:].unsqueeze(-3)),util.transpose_target(get_abs_coord(targets)*inp_dim),hyperparameters['iou_type']))
        iou=iou1.mean().item()
        noobj_box=raw_pred[:,:,4:5].clone()
        noobj_box=helper.uncollapse(noobj_box,mask)
        noobj_mask=helper.uncollapse(noobj_mask.T.unsqueeze(-1),mask)
        
        
        conf=raw_pred[iou_mask.T,:][:,4:5].mean().item()
        class_mask=targets[:,:,5:].type(torch.BoolTensor).squeeze(0)
        if(iou_mask.sum()==class_mask.shape[0]):
            pos_class=raw_pred[iou_mask.T,:][:,5:][class_mask].mean().item()
            neg_class=raw_pred[iou_mask.T,:][:,5:][~class_mask].mean().item()
        else:
            pos_class=0
            neg_class=0
            
        noobj_box=noobj_box[noobj_mask]
        no_obj_conf=noobj_box.mean().item()
        
        raw_pred=raw_pred[iou_mask.T,:]
        anchors=anchors[iou_mask.T,:]
        offset=offset[iou_mask.T,:]
        strd=strd[iou_mask.T,:]
        
        if(strd.shape[0]==sum(mask)):#this means that iou_mask failed and was all true, because max of zeros is true for all lenght of mask strd
            targets[:,:,0:4]=targets[:,:,0:4]*inp_dim
            targets=targets.squeeze(0)
            if hyperparameters['iou_type']==(0,0,0):
                targets[:,0:4]=util.transform_groundtruth(targets,anchors,offset,strd)
                loss=util.yolo_loss(raw_pred,targets,noobj_box,mask,hyperparameters)
                loss.backward()
                optimizer.step()
            else:
                raw_pred=util.transform(raw_pred.unsqueeze(0),anchors.unsqueeze(0),offset.unsqueeze(0),strd.unsqueeze(0)).squeeze(0)
                loss=util.yolo_loss(raw_pred,targets,noobj_box,mask,hyperparameters)
                loss.backward()
                optimizer.step()
            
            avg_conf=avg_conf+conf
            avg_no_conf=avg_no_conf+no_obj_conf
            avg_pos=avg_pos+pos_class
            avg_neg=avg_neg+neg_class
            total_loss=total_loss+loss.item()
            avg_iou=avg_iou+iou
            sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%' ' L:'+ str(loss.item()))
            sys.stdout.write(' IoU:' +str(iou)+' pob:'+str(conf)+ ' nob:'+str(no_obj_conf))
            sys.stdout.write(' PCls:' +str(pos_class)+' ncls:'+str(neg_class))
            sys.stdout.flush()
            prg_counter=prg_counter+1
            train_counter=train_counter+1
        else:
            print('missed')
            print(strd.shape[0])
            prg_counter=prg_counter+1
        iou1=iou1.tolist()
        iou_per_class= [0] * 80
        averager_per_class=[1] * 80
        for i,el in enumerate(iou1):
            name=targets[i,5:].max(0)[1].cpu().detach().numpy()
            iou_per_class[name]=iou_per_class[name]+el
            averager_per_class[name]=averager_per_class[name]+1
        for i in range(80):
            if (iou_per_class[i]!=0):
                if i<40:
                    writer.add_scalar('Iou0/'+str(i), iou_per_class[i]/averager_per_class[i], train_counter)
                else:
                    writer.add_scalar('Iou1/'+str(i), iou_per_class[i]/averager_per_class[i], train_counter)
        writer.add_scalar('AvLoss/train', total_loss/train_counter, train_counter)

        
        writer.add_scalar('AvIoU/train', avg_iou/train_counter, train_counter)

        writer.add_scalar('AvPConf/train', avg_conf/train_counter, train_counter)
        
        writer.add_scalar('AvNConf/train', avg_no_conf/train_counter, train_counter)
        
        writer.add_scalar('AvClass/train', avg_pos/train_counter, train_counter)
        
        writer.add_scalar('AvNClass/train', avg_neg/train_counter, train_counter)
        
        del loss, raw_pred, targets, true_pred, images,iou,noobj_box,conf,iou1
        torch.cuda.empty_cache()
    if misses>0:
        break
#     torch.save(model.state_dict(), PATH)
#     writer.add_scalar('Loss/train', total_loss/train_counter, e)
#     writer.add_scalar('AIoU/train', avg_iou/train_counter, e)
#     writer.add_scalar('PConf/train', avg_conf/train_counter, e)
#     writer.add_scalar('NConf/train', avg_no_conf/train_counter, e)
#     writer.add_scalar('PClass/train', avg_pos/train_counter, e)
#     writer.add_scalar('NClass/train', avg_neg/train_counter, e)
    print('\ntotal number of misses is ' + str(misses))
    print('\n total average loss is '+str(total_loss/train_counter))
    print('\n total average iou is '+str(avg_iou/train_counter))
iou=bbox_iou(get_abs_coord(pred[:,0:4].unsqueeze(0)),get_abs_coord(targets[:,0:4].unsqueeze(0)),(1,0,0))
print(iou)


In [ ]:
import torch
a=torch.tensor([0.93,0.35])
print(torch.softmax(a,dim=0))

In [ ]:
test=torch.tensor([[[1,2],[3,4]]])
print(test.shape[1:])

In [ ]:
print(mask)

idf=torch.stack([targets[:i,5:].sum(axis=0) for i in mask],dim=0)
idf[idf>1]=1

print(idf.sum(axis=0))

In [ ]:

tf=torch.tensor([1/mask[i] for i in range(len(mask)) for j in range(mask[i])]).cuda()
obj_idf=hyperparameters['idf_weights']
print(obj_idf)
classes=targets[:,5:].max(1)[1]
print(classes.tolist())
print(obj_idf['obj_idf'][classes.tolist()])
idf=np.array(obj_idf['obj_idf'][classes.tolist()])
idf=torch.tensor(idf,device='cuda')
idf=-torch.log(idf)

tfidf=tf*idf
tfidf=torch.softmax(tfidf,dim=0)

In [ ]:
targets2,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
transformed_output=true_pred
targets=targets
offset=offset
strd=strd
mask=mask
inp_dim
'''
this function takes the transformed_output and
the target box in respect to the resized image size
and returns a mask which can be applied to select the 
best raw input,anchors and cx_cy_offset
and the noobj_mask for the negatives
targets is a list
'''
#first transpose the centered normalised target coords
centered_target=transpose_target(targets)[:,:,0:2]
#multiply by inp_dim then devide by stride to get the relative grid size coordinates, floor the result to get the corresponding cell
centered_target=torch.floor(centered_target*inp_dim/strd)
#create a mask to find where the gt falls into which gridcell in the grid coordinate system
fall_into_mask=centered_target==offset
fall_into_mask=fall_into_mask[:,:,0]&fall_into_mask[:,:,1]
#     fall_into_mask= ~fall_into_mask
#create a copy of the transformed output
best_bboxes=transformed_output.clone()
#apply reverse mask to copy in order to zero all other bbox locations
best_bboxes[~fall_into_mask]=0   
#transform the copy to xmin,xmax,ymin,ymax
best_responsible_coord=get_abs_coord(best_bboxes)
targets=transpose_target(get_abs_coord(targets))*inp_dim
#calculate best iou and mask
responsible_iou=bbox_iou(best_responsible_coord,targets,True)

responsible_iou[responsible_iou.ne(responsible_iou)] = 0
responsible_mask=responsible_iou.max(dim=0)[0] == responsible_iou

print(responsible_mask.shape)

abs_coord=get_abs_coord(transformed_output)
iou=bbox_iou(abs_coord,targets,True)
iou[iou.ne(iou)] = 0
ignore_mask=0.5<=iou
inverted_mask=iou.max(dim=0)[0] != iou
noobj_mask=~same_picture_mask(responsible_mask.clone()|ignore_mask,mask)

In [ ]:
print(responsible_mask)
if(responsible_mask.sum()>sum(mask)):
    print('jello')
    responsible_mask1=correct_iou_mask(responsible_mask,fall_into_mask)
print(responsible_mask1.shape)

In [ ]:
a=torch.randint(0, 9, (1,)) == torch.arange(9)
a=a.repeat(5)
a=a.repeat(2,1)
print(a)
print(a.T)

In [ ]:
hyperparameters={'lr':0.01,'batch_size':16,'weight_decay':0.001,'optimizer':'sgd','alpha':0.5,'gamma':2,'iou_ignore_thresh':0.5,'tfidf':True,'reduction':'sum'}

In [ ]:
le_mask=(responsible_mask.sum(axis=0)==responsible_mask.sum(axis=0).max())

In [ ]:
responsible_mask[:,le_mask]=(responsible_mask[:,le_mask]&fall_into_mask[le_mask,:].T)

In [ ]:
print(le_mask)
print(fall_into_mask[le_mask,:].shape)


In [ ]:
a=torch.tensor([1,2,3,4,5,6,6,6])
indices=(((a==a.max())==True).nonzero())
for ind in indices:
    print(ind)

In [ ]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes


net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph
cx_cy=net.cx_cy
stride=net.stride

start_time = time.time()
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/pretrained32_precomp_obj_soft.pth'
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if torch.cuda.device_count() > 1:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
    else:
        model=net

    model.to(device)
    
    
    model.load_state_dict(weights)
    model.eval()
except FileNotFoundError: 
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if torch.cuda.device_count() > 1:
      print("Using ", torch.cuda.device_count(), "GPUs!")
      model = nn.DataParallel(net)

    model.to(device)
    model.eval()




print('testing with '+ PATH +'\n')
transformed_dataset=Coco(partition='val',
                                           transform=transforms.Compose([
                                            ResizeToTensor(inp_dim)
                                           ]))



dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=14

dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=False,collate_fn=helper.my_collate, num_workers=4)

true_pos=0
false_pos=0
counter=0
iou_threshold=0.5
confidence=0.01
recall_counter=0
max_detections=None
prg_counter=0
for images,targets,img_name in dataloader:
    inp=images.cuda()
    raw_pred = model(inp, torch.cuda.is_available())
    targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)
    raw_pred=raw_pred.to(device='cuda')
    true_pred=util.transform(raw_pred.clone(),pw_ph,cx_cy,stride)
    
#     classes=true_pred[:,:,5:].max(2)[0]
    objectness=true_pred[:,:,4]
    
    sorted_pred=torch.sort(objectness,descending=True)
    pred_mask=sorted_pred[0]>confidence
    indices=[(sorted_pred[1][e,:][pred_mask[e,:]]) for e in range(pred_mask.shape[0])]
    pred_final=[true_pred[i,indices[i],:] for i in range(len(indices))]
    
    pred_final_coord=[util.get_abs_coord(pred_final[i].unsqueeze(-2)) for i in range(len(pred_final))]
    indices=[nms_box.nms(pred_final_coord[i][0],pred_final[i][:,4],iou_threshold) for i in range(len(pred_final))]
    pred_final=[pred_final[i][indices[i],:] for i in range(len(pred_final))]
#     pred_final[:,0:4]=pred_final[:,0:4]/inp_dim
    helper.write_pred(img_name,pred_final,inp_dim,max_detections)
    sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%')
    prg_counter=prg_counter+1
    
    

print("--- %s seconds ---" % (time.time() - start_time))

# Read txt files containing bounding boxes (ground truth and detections)


In [ ]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
# boundingboxes = helper.getBoundingBoxes()


In [ ]:
from Evaluator import *
from utils import *
from BoundingBox import BoundingBox

from BoundingBoxes import BoundingBoxes


boundingboxes = helper.getBoundingBoxes()
evaluator = Evaluator()

iou=0.5
MMap=0
while iou<1:
    metricsPerClass = evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=iou)
#     print("Average precision values per class:\n")
    # Loop through classes to obtain their metrics
    mAP=0
    counter=0
    for mc in metricsPerClass:
        # Get metric values per each class
        c = mc['class']
        precision = mc['precision']
        recall = mc['recall']
        average_precision = mc['AP']
        ipre = mc['interpolated precision']
        irec = mc['interpolated recall']
        # Print AP per class
        mAP=average_precision+mAP
#         print('%s: %f' % (c, average_precision))
    MMap=MMap+mAP/80
    iou=iou+0.05
    print('\nCOCO map is:',MMap)

In [ ]:
0.6403413112473646
0.6503946905414258
for mc in metricsPerClass:
    c = mc['class']
    average_precision = mc['AP']
    print('%s: %f' % (c, average_precision))

In [ ]:
(0,0,0)==(0,1,0)

In [ ]:
print('%s: %f' % (c, average_precision))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(recall, precision)

In [ ]:
from Evaluator import *
evaluator = Evaluator()
metricsPerClass = evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=0.5)
print("Average precision values per class:\n")
# Loop through classes to obtain their metrics
mAP=0
counter=0
for mc in metricsPerClass:
    # Get metric values per each class
    c = mc['class']
    precision = mc['precision']
    recall = mc['recall']
    average_precision = mc['AP']
    ipre = mc['interpolated precision']
    irec = mc['interpolated recall']
    # Print AP per class
    mAP=average_precision+mAP
    print('%s: %f' % (c, average_precision))

print('map is:',mAP/80)

In [ ]:
0.07290217209136615

In [ ]:
import pandas as pd
coord=pred_final[:,:4].cpu().detach().numpy()
conf=pred_final[:,4:5].cpu().detach().numpy()
mat=np.hstack((conf,coord))

classes=pred_final[:,5:].max(1)[1].cpu().detach().numpy()
classes=np.array([classes]).T

mat=np.hstack((classes,mat))
mat=np.array(mat)

df=pd.DataFrame(mat,index=None,columns=None)
df[0]=df[0].apply(lambda x: int(x))

df.to_csv('test.txt',sep=' ',header=False,index=None)

In [ ]:
np.savetxt('test.txt',mat)

In [ ]:
a=torch.tensor([1,2,2,3,4,2,3,1,4])
print(a.max(0))

In [ ]:
import skimage.io as io

In [ ]:
image=(inp).squeeze(0)
image=np.array(image.cpu())
print(image.shape)
image =  image[:,:,::-1].transpose((1,2,0))
print(image.shape)

In [ ]:
import pandas as pd
import glob

all_files = glob.glob('plots/' + "/*.csv")

li = []

for filename in sorted(all_files):
    print(filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=True)


In [ ]:
df=pd.read_csv('../pointer.csv')
print(df)

In [ ]:
df=df[:None]
print(df)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# gca stands for 'get current axis'

title_list=['AIoU_train','Loss_train','NClass_train','NConf_train','PClass','PConf']

fig = plt.figure(figsize=(30, 10))
fig.suptitle('KL for xy loss')
fig.subplots_adjust(hspace=0.3, wspace=-.6)
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c']
i=2
k=0
while i <18:
    ax = fig.add_subplot(2, 9, i)
    frame.plot(x =1 , y = i,ax=ax,legend=False)
    ax.set_title(title_list[k])
    i=i+3
    k=k+1
plt.savefig('original.png')
plt.show()

In [ ]:
import os
import pandas as pd
import glob
import sys


df = pd.DataFrame(columns=['class','xc','yc','w','h','filename'])
current_dir='/mnt/data1/users/konsa15/workspace/notebooks/coco/labels/coco/yolo'
os.chdir('/mnt/data1/users/konsa15/workspace/notebooks/coco/labels/coco/labels/val2017')
print(os.getcwd())
files = glob.glob("*.txt")
dataset_len=len(files)
prg_counter=0
print(dataset_len)
for file in files:
    with open(file) as f:
        f=f.read()
        box=pd.DataFrame([map(float,x.split()) for x in f.rstrip('\n').split('\n')],columns=['class','xc','yc','w','h'])
        box['filename']=[file for x in f.rstrip('\n').split('\n')]
        df = df.append(box, ignore_index = True)
        
    sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100)+'%')
    prg_counter+=1

print(df)
obj_idf=(df['class'].value_counts(normalize=True).reset_index(name='obj_idf'))
new_df=df.groupby('filename')['class'].value_counts().reset_index(name='count')
img_idf=new_df['class'].value_counts(normalize=True).reset_index(name='img_idf')

obj_idf['img_idf']=img_idf['img_idf']


In [ ]:
df['area']=df['w']*df['h']
total_area=df['area'].sum()

area_idf=df.groupby('class')['area'].sum().reset_index(name='area')
print(area_idf)

In [ ]:
import torch
a=torch.ones(10)
a=a.unsqueeze(1)
a=torch.softmax(a,dim=0)
print(a)

In [ ]:
import os
import pandas as pd
import glob
import sys


from multiprocessing import Pool # for reading the CSVs faster

def my_read_csv(filename):
    try:
        f=open(filename).read()
        box=pd.DataFrame([map(float,x.split()) for x in f.rstrip('\n').split('\n')],columns=['class','xc','yc','w','h'])
        box['filename']=[filename for x in f.rstrip('\n').split('\n')]
        return box
    except FileNotFoundError:
        return None

def load_csvs():
    """Reads and joins all our CSV files into one big dataframe.
    We do it in parallel to make it faster, since otherwise it takes some time.
    Idea from: https://stackoverflow.com/questions/36587211/easiest-way-to-read-csv-files-with-multiprocessing-in-pandas
    
    """
    # set up your pool
    pool = Pool() 
    os.chdir('/mnt/data1/users/konsa15/workspace/notebooks/coco/labels/coco/labels/train2017')
    print(os.getcwd())
    files = os.listdir('.')
    file_list = [filename for filename in files if filename.split('.')[1]=='txt']
    print(len(file_list))
    df_list = pool.map(my_read_csv, file_list)
    # reduce the list of dataframes to a single dataframe
    return pd.concat(df_list, ignore_index=True)

In [ ]:
df=load_csvs()

In [ ]:
df1=df

In [ ]:
# print(df)
obj_idf=(df['class'].value_counts(normalize=True).reset_index(name='obj_freq'))
new_df=df.groupby('filename')['class'].value_counts().reset_index(name='count')
img_idf=new_df['class'].value_counts(normalize=True).reset_index(name='img_freq')

obj_idf['img_freq']=img_idf['img_freq']

total_bins=df['class'].value_counts()
print(total_bins)
yc_idf=df.groupby('class')['yc'].sum()
xc_idf=df.groupby('class')['xc'].sum()

df['area']=df['w']*df['h']
area_idf=df.groupby('class')['area'].sum()


obj_idf['xc']=(xc_idf/total_bins)
obj_idf['yc']=(yc_idf/total_bins)
obj_idf['area']=(area_idf/total_bins)
print(obj_idf)

In [ ]:
import pandas as pd
idf=pd.read_csv('../idf.csv')
print(idf['obj_freq'][[0,0,1,0,1]])

In [ ]:

test=obj_idf
test.plot(x='xc',y='yc',kind='scatter')
# fig1.savefig('corr_with_person_obj.png')
test.plot(x='xc',y='obj_freq',kind='scatter')
test.plot(x='yc',y='img_freq',kind='scatter')

test.plot(x='xc',y='area',kind='scatter')
test.plot(x='yc',y='area',kind='scatter')
# fig2.savefig('corr_with_person_img.png')
test.corr()

In [ ]:
import os
import pandas as pd
import glob
import sys

df = pd.DataFrame()
files = glob.glob("../map/*.csv")
for file in files:
    map_csv=pd.read_csv(file)
    dat1 = pd.DataFrame({file.split('/')[2].split('.')[0]: map_csv['Value']})
    df=pd.concat([df,dat1],axis=1)
#     df.drop(columns=[""])
    df=df.dropna()

df.to_csv('comparative_map.csv')
df.plot()


In [ ]:
df=pd.read_csv('comparative_map.csv')
df=df.drop(columns=["index"])
fig=df.plot().get_figure()
fig.savefig('comparative_map.png')